# A notebook for aumentating image datasets so small they'd fit in a toaster
This is for cropping a Region Of Interest (ROI) but can be modified to fit full image datasets.

In [6]:
import pandas as pd
import pydicom
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
def dicom_to_3_channel(image_path, desired_range=(0, 255)):
    dcm = pydicom.dcmread(image_path)
    phin = dcm.get("PhotometricInterpretation", "Unknown")
    pixel_data = dcm.pixel_array

    if phin == "MONOCHROME2":
        min_value = np.min(pixel_data)
        max_value = np.max(pixel_data)
        pixel_data = ((pixel_data - min_value) / (max_value - min_value)) * (desired_range[1] - desired_range[0]) + desired_range[0]
    else:
        pixel_data = np.max(pixel_data) - pixel_data  # Invert pixel values for MONOCHROME1
        min_value = np.min(pixel_data)
        max_value = np.max(pixel_data)
        pixel_data = ((pixel_data - min_value) / (max_value - min_value)) * (desired_range[1] - desired_range[0]) + desired_range[0]

    if pixel_data.dtype != np.uint8:
        pixel_data = pixel_data.astype(np.uint8)
 
    # If grayscale, replicate into 3 channels
    if len(pixel_data.shape) == 2:
        pixel_data = np.stack((pixel_data,) * 3, axis=-1)

    image = Image.fromarray(pixel_data)
    image_np = np.array(image)

    return image_np

In [ ]:
def find_nxy (xmin, ymin, xmax, ymax, height, width, diff):

    if (xmin - (diff/2)) >= 0 and (ymin - (diff/2)) >= 0:
        if height > width:
            xmin = round (xmin - (diff/2))
            xmax = round (xmax + (diff/2)) 
        else:
            ymin = round (ymin - (diff/2))
            ymax = round (ymax + (diff/2))

    return (xmin,ymin, xmax, ymax)

In [ ]:
def add_zeros (roi, height, width):
    max_dim = max(height, width)
    roi_image = Image.fromarray(roi) # convertir de np array a PIL
    pad = Image.new('RGB', (max_dim, max_dim), (0, 0, 0))
    offset = ((max_dim - roi_image.size[0]) // 2, (max_dim - roi_image.size[1]) // 2)
    pad.paste(roi_image, offset)
    square_roi = np.array(pad) # Convertir de PIL a np array
    return (square_roi)

In [ ]:
def nlm_filter(roi):
    roi_uint8 = roi.astype(np.uint8)
    denoised_roi = cv2.fastNlMeansDenoising(roi_uint8, None, h=4)
    return denoised_roi

## These are the transformations applied. Test with an image

In [ ]:
path = "./Image.dicom"
xmin = 2058
ymin = 1648
xmax = 2470
ymax = 2037

image = dicom_to_3_channel(path)
roi = image[ymin:ymax, xmin:xmax]

roi_image = Image.fromarray(roi)

hflip = roi_image.transpose(Image.FLIP_LEFT_RIGHT)
vflip = roi_image.transpose(Image.FLIP_TOP_BOTTOM)

hflip = np.array(hflip)
vflip = np.array(vflip)

nlmroi = nlm_filter(roi)

roi = roi.astype(float)
min_val = np.min(roi)
max_val =np.max(roi)
roi = 255 * ((roi - min_val) / (max_val - min_val))
froi = roi.astype(np.uint8)

hflip = hflip.astype(float)
min_val = np.min(hflip)
max_val =np.max(hflip)
hflip = 255 * ((hflip - min_val) / (max_val - min_val))
fhflip = hflip.astype(np.uint8)

vflip = vflip.astype(float)
min_val = np.min(vflip)
max_val =np.max(vflip)
vflip = 255 * ((vflip - min_val) / (max_val - min_val))
fvflip = vflip.astype(np.uint8)

nlmroi = nlmroi.astype(float)
min_val = np.min(nlmroi)
max_val =np.max(nlmroi)
nlmroi = 255 * ((nlmroi - min_val) / (max_val - min_val))
fnlmroi = nlmroi.astype(np.uint8)

plt.figure(figsize=[20, 7])
plt.subplot(141); plt.imshow(froi, cmap='gray'); plt.title('OG'); plt.axis('off')
plt.subplot(142); plt.imshow(fhflip, cmap='gray'); plt.title('hflip'); plt.axis('off')
plt.subplot(143); plt.imshow(fvflip, cmap='gray'); plt.title('vflip'); plt.axis('off')
plt.subplot(144); plt.imshow(fnlmroi, cmap='gray'); plt.title('NLM'); plt.axis('off')

## On CPU version

In [ ]:
def augmentation (file, preproc_dir):
    class_assessment_to_folder = {
    ('Architectural distortion', 2): 'ArchDis_BR2',
    ('Architectural distortion', 3): 'ArchDis_BR3',
    ('Architectural distortion', 4): 'ArchDis_BR4',
    ('Architectural distortion', 5): 'ArchDis_BR5',
    ('Calcification', 2): 'Calcification_BR2',
    ('Calcification', 3): 'Calcification_BR3',
    ('Calcification', 4): 'Calcification_BR4',
    ('Calcification', 5): 'Calcification_BR5',
    ('Mass', 2): 'Mass_BR2',
    ('Mass', 3): 'Mass_BR3',
    ('Mass', 4): 'Mass_BR4',
    ('Mass', 5): 'Mass_BR5',
}
    
    df = pd.read_excel (file)

    for i, row in df.iterrows():
        assessment = row['birads']
        name = row['CropFileName']
        img_class = row ['AbnormalityType']
        image_path = row['imagePath']
        index = round(row['Index'])
        xmin = round(row['xmin'])
        ymin = round(row['ymin'])
        xmax = round(row['xmax'])
        ymax = round(row['ymax'])
        height = round(row['h'])
        width = round(row['w'])
        diff = abs (height-width)
        
        if image_path.endswith ('.dcm') or image_path.endswith ('.dicom') or image_path.endswith ('.DCM'): 
            image = dicom_to_3_channel(image_path)
        elif image_path.endswith ('.pgm'):
            image = cv2.imread(image_path, -1)
        else:
            image = cv2.imread(image_path)
            
        if (xmin - (diff/2)) >= 0 and (ymin - (diff/2)) >= 0:
            xmin, ymin, xmax, ymax = find_nxy(xmin, ymin, xmax, ymax, height, width, diff)
            
        roi = image[ymin:ymax, xmin:xmax]
        
        width = xmax - xmin
        height = ymax - ymin

        if width != height:
            roi = add_zeros(roi, height, width)
            
        roi_image = Image.fromarray(roi)

        hflip = roi_image.transpose(Image.FLIP_LEFT_RIGHT)
        vflip = roi_image.transpose(Image.FLIP_TOP_BOTTOM)
        hflip = np.array(hflip)
        vflip = np.array(vflip)

        nlmroi = nlm_filter(roi)
        
        roi = roi.astype(float)
        min_val = np.min(roi)
        max_val =np.max(roi)
        roi = 255 * ((roi - min_val) / (max_val - min_val))
        froi = roi.astype(np.uint8)

        hflip = hflip.astype(float)
        min_val = np.min(hflip)
        max_val =np.max(hflip)
        hflip = 255 * ((hflip - min_val) / (max_val - min_val))
        fhflip = hflip.astype(np.uint8)

        vflip = vflip.astype(float)
        min_val = np.min(vflip)
        max_val =np.max(vflip)
        vflip = 255 * ((vflip - min_val) / (max_val - min_val))
        fvflip = vflip.astype(np.uint8)

        nlmroi = nlmroi.astype(float)
        min_val = np.min(nlmroi)
        max_val =np.max(nlmroi)
        nlmroi = 255 * ((nlmroi - min_val) / (max_val - min_val))
        fnlmroi = nlmroi.astype(np.uint8)
            
        classif_size = (224, 224)
        roi = cv2.resize(froi, classif_size, interpolation=cv2.INTER_LINEAR)
        hflip = cv2.resize(fhflip, classif_size, interpolation=cv2.INTER_LINEAR)
        vflip = cv2.resize(fvflip, classif_size, interpolation=cv2.INTER_LINEAR)
        nlmroi = cv2.resize(fnlmroi, classif_size, interpolation=cv2.INTER_LINEAR)
        
        folder_name = class_assessment_to_folder.get((img_class, assessment), 'WRONG')
        save_dir = os.path.join(preproc_dir, folder_name)
        os.makedirs(save_dir, exist_ok=True)
        
        file_roi = f"{name}_{img_class}_o.png"
        file_roi_path = os.path.join(save_dir, file_roi)
        
        file_hflip = f"{name}_{img_class}_h.png"
        file_hflip_path = os.path.join(save_dir, file_hflip)
        
        file_vflip = f"{name}_{img_class}_v.png"
        file_vflip_path = os.path.join(save_dir, file_vflip)
        
        file_nlm = f"{name}_{img_class}_nlm.png"
        file_nlm_path = os.path.join(save_dir, file_nlm)
        
        # GUARDAR
        cv2.imwrite(file_roi_path, roi)
        cv2.imwrite(file_hflip_path, hflip)
        cv2.imwrite(file_vflip_path, vflip)
        cv2.imwrite(file_nlm_path, nlmroi)
        print (index)  

In [ ]:
augmentation(file="../File.xlsx", preproc_dir="../../Dataset")

## On GPU version

Don't forget switching to cupy_env

In [ ]:
import cupy as cp
from multiprocessing import Pool, cpu_count
from functools import partial

In [ ]:
# Normalize using CuPy
def normalize(img):
    img = cp.array(img, dtype=cp.float32)
    min_val = cp.min(img)
    max_val = cp.max(img)
    img = 255 * ((img - min_val) / (max_val - min_val))
    return cp.asnumpy(img.astype(cp.uint8))

In [ ]:
def augmentation_cuda(file, preproc_dir):
    class_assessment_to_folder = {
        ('Architectural distortion', 2): 'ArchDis_BR2',
        ('Architectural distortion', 3): 'ArchDis_BR3',
        ('Architectural distortion', 4): 'ArchDis_BR4',
        ('Architectural distortion', 5): 'ArchDis_BR5',
        ('Calcification', 2): 'Calcification_BR2',
        ('Calcification', 3): 'Calcification_BR3',
        ('Calcification', 4): 'Calcification_BR4',
        ('Calcification', 5): 'Calcification_BR5',
        ('Mass', 2): 'Mass_BR2',
        ('Mass', 3): 'Mass_BR3',
        ('Mass', 4): 'Mass_BR4',
        ('Mass', 5): 'Mass_BR5',
    }
    
    df = pd.read_excel(file)

    for i, row in df.iterrows():
        assessment = row['birads']
        name = row['CropFileName']
        img_class = row['AbnormalityType']
        image_path = row['imagePath']
        index = round(row['Index'])
        xmin = round(row['xmin'])
        ymin = round(row['ymin'])
        xmax = round(row['xmax'])
        ymax = round(row['ymax'])
        height = round(row['h'])
        width = round(row['w'])
        diff = abs(height - width)
        
        if image_path.endswith(('.dcm', '.dicom', '.DCM')): 
            image = dicom_to_3_channel(image_path)
        elif image_path.endswith('.pgm'):
            image = cv2.imread(image_path, -1)
        else:
            image = cv2.imread(image_path)
        
        if (xmin - (diff / 2)) >= 0 and (ymin - (diff / 2)) >= 0:
            xmin, ymin, xmax, ymax = find_nxy(xmin, ymin, xmax, ymax, height, width, diff)
            
        roi = image[ymin:ymax, xmin:xmax]
        
        width = xmax - xmin
        height = ymax - ymin

        if width != height:
            roi = add_zeros(roi, height, width)
        
        roi_image = Image.fromarray(roi)

        # Apply transformations
        hflip = roi_image.transpose(Image.FLIP_LEFT_RIGHT)
        vflip = roi_image.transpose(Image.FLIP_TOP_BOTTOM)
        hflip = np.array(hflip)
        vflip = np.array(vflip)

        nlmroi = nlm_filter(roi)
        
        roi = cp.array(roi)
        hflip = cp.array(hflip)
        vflip = cp.array(vflip)
        nlmroi = cp.array(nlmroi)
        
        froi = normalize(roi)
        fhflip = normalize(hflip)
        fvflip = normalize(vflip)
        fnlmroi = normalize(nlmroi)
        
        # Convert back to NumPy arrays for OpenCV operations
        froi = cp.asnumpy(froi)
        fhflip = cp.asnumpy(fhflip)
        fvflip = cp.asnumpy(fvflip)
        fnlmroi = cp.asnumpy(fnlmroi)

        classif_size = (224, 224)
        froi = cv2.resize(froi, classif_size, interpolation=cv2.INTER_LINEAR)
        fhflip = cv2.resize(fhflip, classif_size, interpolation=cv2.INTER_LINEAR)
        fvflip = cv2.resize(fvflip, classif_size, interpolation=cv2.INTER_LINEAR)
        fnlmroi = cv2.resize(fnlmroi, classif_size, interpolation=cv2.INTER_LINEAR)
        
        folder_name = class_assessment_to_folder.get((img_class, assessment), 'WRONG')
        save_dir = os.path.join(preproc_dir, folder_name)
        os.makedirs(save_dir, exist_ok=True)
        
        file_roi = f"{name}_{img_class}_o.png"
        file_roi_path = os.path.join(save_dir, file_roi)
        
        file_hflip = f"{name}_{img_class}_h.png"
        file_hflip_path = os.path.join(save_dir, file_hflip)
        
        file_vflip = f"{name}_{img_class}_v.png"
        file_vflip_path = os.path.join(save_dir, file_vflip)
        
        file_nlm = f"{name}_{img_class}_nlm.png"
        file_nlm_path = os.path.join(save_dir, file_nlm)
        
        # Save images
        cv2.imwrite(file_roi_path, froi)
        cv2.imwrite(file_hflip_path, fhflip)
        cv2.imwrite(file_vflip_path, fvflip)
        cv2.imwrite(file_nlm_path, fnlmroi)
        print(index)

In [ ]:
augmentation_cuda(file="./File.xlsx", preproc_dir="././Dataset")

In [52]:
import os
import shutil
import random
import pandas as pd
from collections import defaultdict

In [53]:
def make_dl_folders(dataset, version):
    dl_dirs = {
        'train': os.path.join(version, 'train'),
        'valid': os.path.join(version, 'valid'),
        'test': os.path.join(version, 'test')
    }
    
    # Create destination directories if they don't exist
    for dl_dir in dl_dirs.values():
        os.makedirs(dl_dir, exist_ok=True)
        for subfolder in os.listdir(dataset):
            os.makedirs(os.path.join(dl_dir, subfolder), exist_ok=True)
    
    train_ratio = 0.7
    valid_ratio = 0.2
    
    for subfolder in os.listdir(dataset):
        subfolder_path = os.path.join(dataset, subfolder)
        
        if not os.path.isdir(subfolder_path):
            continue
        
        images = [f for f in os.listdir(subfolder_path) if os.path.isfile(os.path.join(subfolder_path, f))]
        
        # Create a DataFrame for the current subfolder
        df = pd.DataFrame(images, columns=['image'])
        df['subfolder'] = subfolder
        
        # Shuffle the DataFrame
        df = df.sample(frac=1).reset_index(drop=True)
        
        # Calculate the number of images for each category
        total_images = len(df)
        train_count = int(total_images * train_ratio)
        valid_count = int(total_images * valid_ratio)
        
        df['category'] = ['train'] * train_count + \
                         ['valid'] * valid_count + \
                         ['test'] * (total_images - train_count - valid_count)
        
        df = df.sample(frac=1).reset_index(drop=True)
        
        # Copy the images to the respective destination directories
        for index, row in df.iterrows():
            og_path = os.path.join(dataset, row['subfolder'], row['image'])
            dest_path = os.path.join(dl_dirs[row['category']], row['subfolder'], row['image'])
            shutil.copy(og_path, dest_path)
    
    return True


In [54]:
make_dl_folders(dataset="C:/Users/sandra/Documents/Classifier_ResNet50/Calcification/Ver_23052024_PAUG/Dataset", version="C:/Users/sandra/Documents/Classifier_ResNet50/Calcification/Ver_27052024_PAUG")

True

## Merge Dataset 

In [ ]:
def merge_ds(classif_root, output_root):
    for dirpath, dirnames, filenames in os.walk(classif_root):
        for filename in filenames:
            if filename.endswith('.png'):
                # Get the subdirectory name
                subdirectory_name = os.path.basename(dirpath)
                
                # Create a subfolder with the same name in the output directory
                subfolder_path = os.path.join(output_root, subdirectory_name)
                os.makedirs(subfolder_path, exist_ok=True)
                
                # Copy the .png file to the subfolder
                src_path = os.path.join(dirpath, filename)
                dst_path = os.path.join(subfolder_path, filename)
                shutil.copy(src_path, dst_path)